In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import requests

import pandas as pd
import time
from urllib.parse import urlparse
import re


def get_html_content(url):
    # set Chrome options to set language to English
    options = Options()
#     options.add_argument("--lang=en-US")
    
    prefs = {
      "translate_whitelists": {"ja":"en"},
      "translate":{"enabled":"true"}
    }
    options.add_experimental_option("prefs", prefs)
    # initialize the webdriver with options
#     driver = webdriver.Chrome(executable_path='C:/Users/najmus/Documents/chromedriver/chromedriver.exe', options=options)
    driver = webdriver.Chrome(executable_path='/Users/pcworld/chromedriver/chromedriver', options=options)
    
    # send a request to the URL
    driver.get(url)
    
    # retrieve the page source
    html_content = driver.page_source
    
    # close the webdriver
    driver.quit()
    
    # return the HTML content
    return html_content


In [5]:
def get_divs_with_class(response, class_name):
    # parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response, 'html.parser')
    
    # find all div elements with the specified class
    divs = soup.find_all('div', {'class': class_name})
    
    products = []
    
    for div in divs:
        products.append(div)
    
    # return the list of attribute values
    return products

In [6]:
def get_anchor_with_class(response, class_name, attribute):
    # parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response, 'html.parser')
    
    # find all div elements with the specified class
    divs = soup.find_all('div', {'class': class_name})
    
    products_id = []
    
    for div in divs:
        anchor = div.find('a')
        if anchor:
            products_id.append(anchor.get(attribute))
    
    # return the list of attribute values
    return products_id

In [ ]:
response = get_html_content('https://shop.adidas.jp/item/?gender=mens&category=footwear&order=1&page=1')




In [ ]:
href = get_anchor_with_class(response,'itemCardArea-cards test-card css-dhpxhu','href')
products = get_divs_with_class(response,'itemCardArea-cards test-card css-dhpxhu')
# products_id = get_divs_with_class(response,'itemCardArea-cards test-card css-dhpxhu','data-ga-eec-product-id')

In [ ]:
print(href)
# len(products_id)

In [ ]:
len(href)

In [ ]:
# count = 1
# for p in products:
#     print(str(count)+ '--' + str(p))
#     print('--------------------------------------------------------------------')
#     count+=1

In [7]:
def make_product_link(link,base_url_product,items_count):
    page_count = 1
    item = 1
    total_product_count = 0
    base = base_url_product
    product_url = []
    while total_product_count < items_count:
        
        if page_count % 11 == 0:
            page_count = 1
        link = link[:-1]
        link = link + str(page_count)
        response = get_html_content(link)
        paths = get_anchor_with_class(response,'itemCardArea-cards test-card css-dhpxhu','href')
        for path in paths:
            full_link = base + path
            print(full_link)
            
            product_url.append(full_link)
            if len(product_url) == items_count:
                break
            
        total_product_count = len(product_url)
        
        print(link)
        page_count+=1
        item +=1
        
    return product_url
    
    

In [8]:
link = 'https://shop.adidas.jp/item/?gender=mens&category=wear&group=outer&page=3'
product_url = make_product_link(link,'https://shop.adidas.jp',5)

/Users/pcworld/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


https://shop.adidas.jp/products/HS2081/
https://shop.adidas.jp/products/HY2728/
https://shop.adidas.jp/products/HR4219/
https://shop.adidas.jp/products/HK7513/
https://shop.adidas.jp/products/HR8692/
https://shop.adidas.jp/item/?gender=mens&category=wear&group=outer&page=1


In [9]:
product_url
len(product_url)

5

In [115]:
def parse_sizes(html):
    sizes = []
    soup = BeautifulSoup(html, 'html.parser')
    size_buttons = soup.find_all('button', {'type': 'button'})
    for button in size_buttons:
        if 'disable' not in button.get('class', []):
            size = button.text.strip()
            sizes.append(size)
    return sizes

def parse_path(html):
    sizes = []
    soup = BeautifulSoup(html, 'html.parser')
    paths = soup.find_all('a', {'class': 'breadcrumbListItemLink'})
    p = '/'
    for i, path in enumerate(paths):
        if i != 0:  # Skip the first path
            size = path.text.strip()
            p = p + str(size) + '/'
            sizes.append(size)
    return p

def get_element(soup,tag,class_name):
    soup = soup
    if soup:
        element = soup.find(tag, {'class': class_name})
        element_text = element.text.strip() if tag else None
    else:
        element_text = None
    
    return element_text

def driver_config():
    
    options = Options()
    options.add_argument("--lang=en-US")

    prefs = {
      "translate_whitelists": {"ja":"en"},
      "translate":{"enabled":"true"}
    }
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(executable_path='/Users/pcworld/chromedriver/chromedriver', options=options)
    time.sleep(15)
    
    return driver

def scroll_and_delay(driver):
    driver = driver
    time.sleep(5)
    
    driver.execute_script("window.scrollBy(0, 1200)")
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 300)")
    
#     element = driver.find_element("id", "BVRRContainer")
#     driver.execute_script("arguments[0].scrollIntoView();", element)
    time.sleep(5)
    driver.execute_script("window.scrollBy(0, 300)")
    time.sleep(2)

#     driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
#     for i in range(10):
#         driver.execute_script('window.scrollBy(0, 200);')
#         time.sleep(1)
#     element = driver.find_element_by_xpath('//div[contains(text(),"PRODUCT REVIEW")]')
#     driver.execute_script("arguments[0].scrollIntoView();", element)
    
    element = driver.find_element("id", "BVRRContainer")
    driver.execute_script("arguments[0].scrollIntoView();", element)
    
    time.sleep(3)

def sizechart(soup):

    soup = soup
    try:
        size_chart_div = soup.find("div", class_="sizeChart test-sizeChart css-l7ym9o")
        table = size_chart_div.find("table", class_="sizeChartTable")
        thead = table.find("thead", class_="sizeChartTHeader")
        header_row = thead.find_all("tr")[1]

        headers = []

        for th in table.find_all('th', {'class': True }):
            if th.text:
                headers.append(th.text.strip())
            else:
                headers.append('size')

        tbody = size_chart_div.find("tbody")
        rows = []
        for tr in tbody.find_all("tr"):
            row_data = [td.text.strip() for td in tr.find_all("td")]
            row_data = '-'.join(row_data).replace(" ", "").replace(".", "")
            rows.append(row_data)

        data = []
        data.append(str(headers))
        data.append(str(rows))
    except:
        data = ['null','null']
#     df = pd.DataFrame([data], columns=header)
    
    return data
def review_parser(soup,p_id):
#     soup = BeautifulSoup(html, 'html.parser')
    soup = soup
    review = {}
    df = []
    
    review['p_id'] = p_id
    df.append(p_id)
    
    rating = soup.find('div', {'class': 'BVRRRatingNormalImage'}).img['alt']
#     alt_value = soup.find('div', {'class': 'BVRRRatingNormalImage'}).img['alt']

    review['rating'] = rating
    df.append(rating)

    title = soup.find('span', {'class': 'BVRRReviewTitle'}).text
    review['title'] = title
    df.append(title)

    date = soup.find('span', {'class': 'BVRRReviewDate'}).text
    review['date'] = date
    df.append(date)

    full_review = soup.find('div', class_='BVRRReviewText').text
    review['full_review'] = full_review
    df.append(full_review)
    
    
    user = soup.find('span', class_='BVRRValue BVRRUserNickname').text
    review['user'] = user
    df.append(user)
    
    recommendations = soup.find('div', class_='BVRRReviewDisplayStyle5Recommendations').text
    review['recommendations'] = recommendations
    df.append(recommendations)
    
    return review

def get_image_url(p_id,soup):
    soup=soup
    image_srcs = []
    df = {}
    df['p_id']=p_id
    soup = soup.find('ul', {'class': 'slider-list test-slider-list'})
    for img in soup.find_all('img'):
        src = img.get('src')
        if src:
            image_srcs.append('https://shop.adidas.jp'+ src)
    df['resource_url'] = str(image_srcs)
    return df

#     try:
#         image_urls = soup.find_all('ul', {'class': 'slider-list test-slider-list'}).img['src']
#         for src in image_urls:
#             img_srcs.append(src)
#     except:
#         image_urls = soup.find('ul', {'class': 'slider-list test-slider-list'}).img['src']
#         img_srcs.append(image_urls)
        
#     return img_srcs

def click_all_tabs(driver):
    driver = driver
#     time.sleep(10)
#     driver.get(link)  # load the HTML code in the browser
    tab_labels = driver.find_elements(By.CLASS_NAME, 'tabLink')
    driver.execute_script("window.scrollBy(0, 600)")
    for label in tab_labels:
#         data_href = label.get_attribute('data-href')
        print(label)
        label.click()  # click the tab label
        time.sleep(8)  # pause for 5 seconds
#         WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, f"//div[@data-href='{data_href}']")))
#     driver.quit()  # close the browser

def parse_span_class(html):
    pattern = r'mod-marker_(\d+_\d+)'
    match = re.search(pattern, html)
    if match:
        return match.group(1)
    else:
        return None
    
def parse_description_text(p_id,soup):
    soup = soup
    product_description = {}
    product_description['p_id']=p_id
    heading = soup.find('h4', class_='heading itemFeature test-commentItem-subheading')
    part1 = soup.find('div', class_='description_part details test-itemComment-descriptionPart')
    part2 = soup.find('ul', class_='articleFeatures description_part css-1lxspbu')
    product_description['Title_Description']=heading.text.strip()
    product_description['Detailed_Description']=part1.text.strip()
    product_description['Summary_Description']=part2.text.strip()
#     text = part1.text.strip()
    return product_description

def parse_tags(soup):
    soup = soup
    tags = []
    soup = soup.find('div',{'class':'itemTagsPosition'})
    for link in soup.find_all('a'):
        tag = link.text.strip()
        if tag:
            tags.append(tag)
    return ', '.join(tags)

def parse_product_info(soup):
    soup = soup
    product_list = []
    items = soup.find_all('div', {'class': 'coordinate_box'})
    for item in items:
        for li in item.find_all('li'):
            product = {}
            a = li.find('div', {'class': 'coordinate_item_tile test-coordinate_item_tile'})
            product['name'] = a.img['alt']
            product['price'] = a['data-price']
            product['id'] = a['data-articleid']
            product['link'] = f"https://shop.adidas.jp/{a['data-articleid']}" + "/"
            product_list.append(product)
    return product_list


In [116]:
def get_h1_from_links(links, class_name,span_class,salesst,size,c_path):
  
    #------------Driver Config-----------start---------

    driver = driver_config()
    
    #------------Driver Config-----------end---------

    data = []
    reviews_df = pd.DataFrame(columns=['p_id', 'rating', 'title', 'date', 'full_review', 'user', 'recommendations'])
    product_description = pd.DataFrame(columns=['p_id','Title_Description', 'Detailed_Description', 'Summary_Description'])
    resources = pd.DataFrame(columns=['p_id', 'resource_url'])
    # Loop through each link and retrieve the h1 element with the specified class
    for link in links:
        
        try:
            driver.get(link) # Navigate to the link
            html_content = driver.page_source # Retrieve the page source
            soup = BeautifulSoup(html_content, 'html.parser') # Parse the page source with BeautifulSoup
            if soup.find('div', {'class': 'errorMessage test-errorMessage'}):
                continue        
        except:
            continue
            
#         scroll_and_delay(driver)
#         driver.execute_script("window.scrollBy(0, 300)")
        click_all_tabs(driver)
        
        html_content = driver.page_source # Retrieve the page source
    
        soup = BeautifulSoup(html_content, 'html.parser') # Parse the page source with BeautifulSoup
        
        #-------------Getting Product Info------------start------------------
        parsed_url = urlparse(link)
        product_id = parsed_url.path.split('/')[-2]
        
        description = parse_description_text(product_id,soup)
        product_description = product_description.append(description, ignore_index=True)
        
#         print(product_description)
        cordinate_items = parse_product_info(soup)
        print(cordinate_items)
        
        product_review = soup.find('div', {'class': 'BVRRDisplayContentBody'})
#         review = review_parser(product_review)
        
        if product_review:
            array = []
            for r in product_review:
#                 head_review = review.find('div',{'class':'BVRRReviewDisplayStyle5Header'})
#                 print(head_review.text)
                review = review_parser(r,product_id)
                reviews_df = reviews_df.append(review, ignore_index=True)

                array.append(str(review))
#                 print(review)
#                 print('------------------------------')
#             print(array)
            
        product_size_chart = sizechart(soup)#SIZECHART
        product_name = get_element(soup,'h1',class_name)#NAME
        category_name = get_element(soup,'span','categoryName test-categoryName')
        product_price = get_element(soup,'div',span_class)#PRICE
        product_status = get_element(soup,'div',salesst)#SALESSTATUS
        
        #SIZE
        size_selector = soup.find('ul', {'class': size})
        sizes = parse_sizes(str(size_selector))
        
        #SIZE Sense
#         if soup.find('div',{'class':'sizeFitBar css-zrdet1'}):
#             size_sense = parse_span_class(str(soup.find('div',{'class':'bar'})))
# #             print(str(size_sense))
# #             print(str(soup.find('div',{'class':'sizeFitBar css-zrdet1'})))
#         else:
#             print('not available')
        #BREADCRUM WRAP
        category_path = soup.find('div', {'class': c_path})
        path = parse_path(str(category_path))
        
        try:
            rating_container = soup.find('div', {'class': 'BVRRQuickTakeCustomWrapper'})
            rating = rating_container.find('div', {'class': 'BVRRRatingNormalImage'}).img['alt']
    #         rating = get_element(rating_container,'span','BVRRRatingNormalImage') #BVRRRatingNormalImage
            recommended_rate = get_element(rating_container,'span','BVRRBuyAgainPercentage')        
        except:
            rating = 'Yet to be reviewed'
            recommended_rate = 'Yet to be recommended'
        
        image_urls = get_image_url(product_id,soup)
        resources = resources.append(image_urls, ignore_index=True)
        
        KWs = parse_tags(soup)
#         print(KWs)

#         print(image_urls)
        #-------------Getting Product Info------------end------------------
        
#         print(product_size_chart)
        # Add the link, h1 text, and span text to the data list
        data.append([link,product_id,category_name, product_name, product_price,product_status,str(sizes),str(path),str(product_size_chart[0]),str(product_size_chart[1]),str(rating),str(recommended_rate),str(KWs)])
    
    # Close the webdriver
    driver.quit()
    
    # Convert the data list to a Pandas DataFrame and return it
    df = pd.DataFrame(data, columns=['Link','Product_id','Category_Name' ,'Name', 'Price','Product_Status','Available_Sizes','Path','Size_Attributes','Size_Attributes_Values','Rating','Recommended_rate','KWs'])
    return df,reviews_df,product_description,resources


In [117]:
dataframe,review,product_description,resources = get_h1_from_links(product_url, 'itemTitle test-itemTitle','articlePrice test-articlePrice css-1apqb46','articleSalesStatus css-prnrg3','sizeSelectorList','breadcrumb_wrap')

/Users/pcworld/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


<selenium.webdriver.remote.webelement.WebElement (session="1721014d0c41fed8d0a5339c252544b3", element="aefa8cf4-3d6d-4eb1-acc4-b3b4b2060433")>
<selenium.webdriver.remote.webelement.WebElement (session="1721014d0c41fed8d0a5339c252544b3", element="cbb9af25-bf88-451e-a1e7-8b826cf53618")>
<selenium.webdriver.remote.webelement.WebElement (session="1721014d0c41fed8d0a5339c252544b3", element="b0aa61c5-55fd-4353-8532-2528ef3090ee")>
<selenium.webdriver.remote.webelement.WebElement (session="1721014d0c41fed8d0a5339c252544b3", element="f01bcdf4-8401-4197-83c2-4a712c7affb6")>
[{'name': 'Adicolor Classics Waffle Beckenbauer Track Pants', 'price': '11,000', 'id': 'HS2079', 'link': 'https://shop.adidas.jp/HS2079/'}]
<selenium.webdriver.remote.webelement.WebElement (session="1721014d0c41fed8d0a5339c252544b3", element="ed2f9e4e-b95b-4bbd-ad69-16d50bde5fb7")>
<selenium.webdriver.remote.webelement.WebElement (session="1721014d0c41fed8d0a5339c252544b3", element="381abf93-a9bf-4cd3-9c7a-793f6d7ebf02")>
<s

In [89]:
dataframe

,Link,Product_id,Category_Name,Name,Price,Product_Status,Available_Sizes,Path,Size_Attributes,Size_Attributes_Values,Rating,Recommended_rate,KWs
0,https://shop.adidas.jp/products/HS2081/,HS2081,originals,Adicolor Classics Waffle Beckenbauer Track Jacket,"12,100",NEW,"['M.', 'L.', 'XL']",/mens/wear/clothes/outer/Jacket/originals/Beck...,"['size', 'chest circumference', 'Back length',...","['XS-S-M-L-XL-2XL-3XL', '95cm-98cm-103cm-105cm...",5/5,100%,"wear/clothes, outer, Jacket, Jersey, originals..."
1,https://shop.adidas.jp/products/HY2728/,HY2728,originals,Skateboarding Firebird Truck Jacket (Gender Ne...,"12,100",,"['XS', 'S.', 'M.', 'L.', 'XL', '2XL', '3XL']",/unisex/skateboarding/wear/clothes/outer/wind ...,"['size', 'chest circumference', 'Back length',...","['3XS-2XS-XS-S-M-L-XL-2XL-3XL', '100cm-104cm-1...",4.1/5,80%,"wear/clothes, outer, wind breaker, originals, ..."
2,https://shop.adidas.jp/products/HR4219/,HR4219,originals,adidas adventure premium windbreaker,"16,500",NEW,"['2XS', 'XS', 'S.', 'M.', 'L.', 'XL', '2XL', '...",/mens/wear/clothes/outer/wind breaker/original...,"['size', 'chest circumference', 'Back length',...","['2XS-XS-S-M-L-XL-2XL-3XL', '108cm-113cm-116cm...",Yet to be reviewed,Yet to be recommended,"wear/clothes, outer, wind breaker, originals, ..."
3,https://shop.adidas.jp/products/HK7513/,HK7513,originals,Essentials+ Dye Coach Shirt,"9,240",,"['2XS', 'XS', 'S.', 'M.', 'L.', 'XL', '2XL', '...",/mens/wear/clothes/outer/Jacket/originals/Esse...,"['size', 'chest circumference', 'Back length',...","['2XS-XS-S-M-L-XL-2XL-3XL', '112cm-117cm-120cm...",4.8/5,100%,"wear/clothes, outer, Jacket, originals, better..."
4,https://shop.adidas.jp/products/HR8692/,HR8692,originals,Adicolor Neuclassics Windbreaker,"14,300",NEW,"['XS', 'S.', 'M.', 'L.', 'XL', '2XL']",/mens/wear/clothes/outer/wind breaker/original...,"['size', 'chest circumference', 'Back length',...","['XS-S-M-L-XL-2XL', '117cm-120cm-125cm-127cm-1...",Yet to be reviewed,Yet to be recommended,"wear/clothes, outer, wind breaker, originals, ..."


In [90]:
review
product_description
resources

,p_id,resource_url
0,HS2081,['https://shop.adidas.jp/photo/HS/HS2081/s-HS2...
1,HY2728,['https://shop.adidas.jp/photo/HY/HY2728/s-HY2...
2,HR4219,['https://shop.adidas.jp/photo/HR/HR4219/s-HR4...
3,HK7513,['https://shop.adidas.jp/photo/HK/HK7513/s-HK7...
4,HR8692,['https://shop.adidas.jp/photo/HR/HR8692/s-HR8...


In [91]:
writer = pd.ExcelWriter('adidas_japan.xlsx')

# write each dataframe to a different sheet
dataframe.to_excel(writer, sheet_name='product_details', index=False)
review.to_excel(writer, sheet_name='product_reviews', index=False)
product_description.to_excel(writer, sheet_name='product_description', index=False)
resources.to_excel(writer, sheet_name='resources', index=False)

# save the writer
writer.save()

In [ ]:
def parse_html_website(url):
    # Send a request to the website and parse the HTML with BeautifulSoup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all the div elements with class attributes and extract their text and class names
    divs = soup.find_all('div', class_=True)
    div_text = []
    div_class = []
    
    # Loop through each div element and check if it has any text
    for div in divs:
        text = div.get_text(strip=True)
        if text:
            div_text.append(text)
            div_class.append(div['class'])
    
    # Create a Pandas DataFrame from the div text and class lists
    df = pd.DataFrame({'div_text': div_text, 'div_class': div_class})
    
    return df

In [ ]:
df = parse_html_website('https://shop.adidas.jp/products/HC7132/')

In [ ]:
df

In [ ]:
print(df)

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

url = """<div class="sizeChart test-sizeChart css-l7ym9o"><table class="sizeChartTable"><thead class="sizeChartTHeader"><tr class="sizeChartTRow"><th class="sizeChartTHeaderCell test-combined_table_header"></th></tr><tr class="sizeChartTRow"><th class="sizeChartTHeaderCell test-combined_table_header"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">chest circumference</font></font></th></tr><tr class="sizeChartTRow"><th class="sizeChartTHeaderCell test-combined_table_header"><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">Back length</font></font></th></tr></thead></table><table class="sizeChartTable"><tbody><tr class="sizeChartTRow"><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">XS</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">S.</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">M.</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">L.</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">O (XL)</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">XO (2XL)</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">2XO (3XL)</font></font></span></td></tr><tr class="sizeChartTRow"><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">98cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">102cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">106cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">110cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">114cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">118cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">122cm</font></font></span></td></tr><tr class="sizeChartTRow"><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">64cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">66cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">68cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">70cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">72cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">74cm</font></font></span></td><td class="sizeChartTCell"><span><font style="vertical-align: inherit;"><font style="vertical-align: inherit;">76cm</font></font></span></td></tr></tbody></table></div>"""


def sizechart(url):

    driver = driver_config()

    # send a request to the URL
    driver.get(url)
    
    scroll_and_delay(driver)

    # retrieve the page source
    html_content = driver.page_source
    
    # close the webdriver
    driver.quit()
    
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')

    # Create a BeautifulSoup object with the response text and specify the parser
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the div containing the table and extract the table data
#     size_chart_div = soup.find('div', {'class' : 'sizeChart test-sizeChart css-l7ym9o' })

    size_chart_div = soup.find("div", class_="sizeChart test-sizeChart css-l7ym9o")
    table = size_chart_div.find("table", class_="sizeChartTable")
    thead = table.find("thead", class_="sizeChartTHeader")
    header_row = thead.find_all("tr")[1]
#     headers = [th.text.strip() for th in header_row.find_all("th")[1:]]
    
    headers = []
    for th in table.find_all('th', {'class': True }):
        if th.text:
            headers.append(th.text.strip())
        else:
            headers.append('size')

#     table = soup.find("table", class_="sizeChartTable")
#     thead = table.find("thead", class_="sizeChartTHeader")
#     header_row = thead.find_all("tr")
#     headers = [th.text.strip() for th in header_row.find_all("th")]
#     print(size_chart_div)
#     print('-----')
#     print(table)
#     print('-----')
#     print(thead)
#     print('-----')
#     print(header_row)
#     print('-----')
#     print(headers)

    tbody = size_chart_div.find("tbody")
    rows = []
    for tr in tbody.find_all("tr"):
        row_data = [td.text.strip() for td in tr.find_all("td")]
        row_data = '-'.join(row_data).replace(" ", "").replace(".", "")
        rows.append(row_data)
    
    data = []
    data.append(str(headers))
    data.append(str(rows))
#     print('-----')
#     print(rows)
#     print('-----')
#     print(data)
    header = ['attributes','values']
#     # Create a DataFrame with the extracted table data
    df = pd.DataFrame([data], columns=header)
#     df.set_index(headers[0], inplace=True)
    
    return data

In [ ]:
url = 'https://shop.adidas.jp/products/HE7194/'
df = sizechart(url)

In [ ]:
print(df)

In [ ]:
url = 'https://shop.adidas.jp/products/IA3448/'
df1 = sizechart(url)

In [ ]:
print(df1)

In [ ]:
df

In [ ]:
# merged_df = pd.merge(df, df1, on='attributes', how='outer')
# dfs = [df,df1]

# merged_df = pd.concat(dfs, axis=1, sort=False)
merged_df = pd.concat([df, df1], axis=0, ignore_index=True)

In [ ]:
# merged_df = pd.merge(merged_df, df, on='size', how='outer')
merged_df

In [ ]:
merged_df

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def click_all_tabs(link):
    driver = driver_config()
    driver.get(link)  # load the HTML code in the browser
    tab_labels = driver.find_elements(By.CLASS_NAME, 'tabLink')
    driver.execute_script("window.scrollBy(0, 600)")
    for label in tab_labels:
        data_href = label.get_attribute('data-href')
        label.click()  # click the tab label
        time.sleep(5)  # pause for 5 seconds
    driver.quit()  # close the browser


In [ ]:
click_all_tabs('https://shop.adidas.jp/products/HS2081/')

In [50]:
import re

def parse_span_class(html):
    pattern = r'mod-marker_(\d+_\d+)'
    match = re.search(pattern, html)
    print(type(match.group(1)))
    if match:
        return match.group(1)
    else:
        return None


In [53]:
html = '<div class="bar"><span class="test-marker marker mod-marker_1_5"></span><ul class="content"><li></li><li></li><li></li><li></li><li></li></ul></div>'
span_class = parse_span_class(html)
print(span_class)  # Output: 2_5


<class 'str'>
1_5


In [99]:
def parse_product_info(soup):
    soup = soup
    product_list = []
    items = soup.find('div', {'class': 'coordinate_box'})
    for item in items:
        for li in item.find_all('li'):
            product = {}
            a = li.find('a', {'class': 'coordinate_item_tile'})
            product['name'] = a['data-articleid']
            product['price'] = a['data-price']
            product['id'] = a['data-articleid']
            product['link'] = f"https://www.example.com/products/{a['data-articleid']}"
            product_list.append(product)
    return items

In [100]:
response = requests.get('https://shop.adidas.jp/products/HY2728/')

# Create a BeautifulSoup object with the response text and specify the parser
soup = BeautifulSoup(response.text, "html.parser")
output = parse_product_info(soup)

In [101]:
output

<div class="coordinate_box"><div class="coordinateItems css-vjspya"><div class="item_tile_wrapper test-item_tile_wrapper clearfix"><div class="css-1dfrz3r"></div></div></div><div class="coordinate_item_container test-coordinate_item_container"></div></div>

In [95]:
response.text

'<!DOCTYPE html><html lang="ja"><style data-emotion="css-global 1jykadx">@font-face{font-family:adiHaus;src:url(/static/fonts/AdiHaus-Regular.woff2) format(\'woff2\'),url(/static/fonts/AdiHaus-Regular.woff) format(\'woff\'),url(/static/fonts/AdiHaus-Regular.ttf) format(\'truetype\');font-weight:400;font-style:normal;font-display:swap;}@font-face{font-family:adiHaus;src:url(/static/fonts/AdiHaus-Bold.woff2) format(\'woff2\'),url(/static/fonts/AdiHaus-Bold.woff) format(\'woff\'),url(/static/fonts/AdiHaus-Bold.ttf) format(\'truetype\');font-weight:700;font-style:normal;font-display:swap;}@font-face{font-family:adihausDIN-cond;src:url(/static/fonts/AdihausDIN-CondIta.woff2) format(\'woff2\'),url(/static/fonts/AdihausDIN-CondIta.woff) format(\'woff\'),url(/static/fonts/AdihausDIN-CondIta.ttf) format(\'truetype\');font-weight:400;font-style:italic;font-display:swap;}@font-face{font-family:adiNeuePro;src:url(/static/fonts/adineuepro-regular-webfont.woff) format(\'woff\'),url(/static/fonts/adi